# MasterMonitor Sensor Script

## AdaFruit IO

### Credentials

In [2]:
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

load_dotenv()

AdaFruitIO_Username = os.environ.get("ADAFRUIT_IO_USERNAME")
AdaFruitIO_Key = os.environ.get("ADAFRUIT_IO_KEY")

aio = Client(AdaFruitIO_Username, AdaFruitIO_Key)

### Feeds

In [ ]:
def getFeed(name):
    try:
        feedID = aio.feeds(name)
    except RequestError:
        feed = Feed(name=name)
        feedID = aio.create_feed(feed)
    return feedID

soundFeed = getFeed("sound")
luxFeed = getFeed("light")
intTempCfeed = getFeed("inttempc")
intTempFfeed = getFeed("inttempf")
extemp2Feed = getFeed("extemp2")
extempFeed = getFeed("extemp")
humidFeed = getFeed("humidity")
pressFeed = getFeed("pressure")
tvocsFeed = getFeed("new-sensor.tvocs")
co2Feed = getFeed("new-sensor.co2")

## Google Sheets

In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "enviro-pi_log"

scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).worksheet("Values")

## BME280 Sensor
###### (Humidity, Pressure, and Temperature)

In [ ]:
from bme280 import BME280

bme280 = BME280()

inter.c = bme280.get_temperature()
inter.f = InternalC * 1.8 + 32
humid = bme280.get_humidity()
press = bme280.get_pressure()

## External Temperature Sensor

In [5]:
import smbus

def readTemp():
    address = 0x48
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

ext.c = readTemp()
ext.f = External * 1.8 + 32

## CCS811 Sensor
###### CO2 and VOCs

In [7]:
import busio
import board
import adafruit_ccs811

i2c = board.I2C()
ccs =  adafruit_ccs811.CCS811(i2c)

while (not ccs.data_ready):
    pass

print ("CCS811 Sensor ready.")

co2 = ccs.eco2
tvocs = ccs.tvoc

CCS811 Sensor ready.


## Light Sensor

In [ ]:
from ltr559 import LTR559

ltr559 = LTR559()

lux = ltr559.get_lux()
prox = ltr559.get_proximity()

## Noise Sensor

In [ ]:
from enviroplus.noise import Noise

noise = Noise()

range=[(100, 1200)]
amps = noise.get_amplitudes_at_frequency_ranges(range)

## Main

### Imports

In [8]:
import ST7735

import time
from IPython.display import clear_output
from board import *
from datetime import datetime

### Variables

In [9]:
try:
    metadata = {'lat': 36.017433,
           'lon': -84.239815,
           'ele': 0,
           'created_at': None}

    time_calculation = time.time()
    
    aio.send_data(co2Feed.key, co2, metadata)
    aio.send_data(tvocsFeed.key, tvocs, metadata)
    aio.send_data(soundFeed.key, amps[0], metadata)
    aio.send_data(luxFeed.key, lux, metadata)
    aio.send_data(intTempCfeed.key, inter.c, metadata)
    aio.send_data(intTempFfeed.key, inter.f, metadata)
    aio.send_data(extemp2Feed.key, ext.f, metadata)
    aio.send_data(extempFeed.key, ext.c, metadata)
    aio.send_data(humidFeed.key, humid, metadata)
    aio.send_data(pressFeed.key, press, metadata)
    
    time_elapsed = time.time() - time_calculation

except IOError:
        pass
    
except KeyboardInterrupt:
    print("Terminated by User")
    break

except RuntimeError:
    print("Not Good")

### Loop

In [10]:
while True:
    try:
        out = "External-Temp: {0:.2f} C, {1:.2f} F - Internal-Temp: {2:.2f} C, {3:.2f} F, - Humidity: {4:.2f} % - Pressure: {5:.2f} hPa".format(ext.c, ext.f, inter.c, inter.f, humid, press)
        _ = "Light: {0:.2f} - Noise: {1:.2f} - Proximity: {2:.2f}".format(lux, amps[0], prox)
        put = "CO2: {6:.2f} ppm, - TVOCs: {7:.2f} ppb".format(co2, tvocs)
        print(out, _, put)
        print('Updated: ', datetime.now())
        
        now = datetime.now()
        nowtime = now.strftime("%H:%M:%S")
        nowday = now.strftime("%m/%d/%Y")
        row = [nowday, nowtime, inter.c, inter.f, ext.c, ext.f, humid, press, lux, amps[0], co2, tvocs]
        sheet.append_row(row)
        
        time.sleep(30)
        clear_output()
        time_elapsed = time.time() - time_calculation
        
        if time_elapsed == 5:
            time_calculation = time.time()
            aio.send_data(co2Feed.key, co2, metadata)
            aio.send_data(tvocsFeed.key, tvocs, metadata)
            aio.send_data(soundFeed.key, amps[0], metadata)
            aio.send_data(luxFeed.key, lux, metadata)
            aio.send_data(intTempCfeed.key, inter.c, metadata)
            aio.send_data(intTempFfeed.key, inter.f, metadata)
            aio.send_data(extemp2Feed.key, ext.f, metadata)
            aio.send_data(extempFeed.key, ext.c, metadata)
            aio.send_data(humidFeed.key, humid, metadata)
            aio.send_data(pressFeed.key, pressure, metadata)
            
            print("Sending Data to the AdaFruit Cloud......")

    except IOError:
        pass
    
    except KeyboardInterrupt:
        print("Loop Terminated by User")
        break
        
    except RuntimeError:
        print("Not Good")

Temperature: External: 21.75 C, External: 71.15 F, Internal: 28.59 C, Internal: 83.46 F, - Humidity: 50.88 % - Pressure: 990.65 hPa, - CO2: 400.00 ppm, - TVOCs: 0.00 ppb
Loop Terminated by User
